In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

In [3]:
NUM_CLASSES = 5  # Number of classes (eye diseases)
IMG_SIZE = (224, 224)  # Input image size expected by ResNet
BATCH_SIZE = 16
EPOCHS = 20

In [4]:
train_dir = '/content/drive/MyDrive/archive/train'
test_dir = '/content/drive/MyDrive/archive/test'


In [5]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [6]:
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

Found 327 images belonging to 5 classes.
Found 56 images belonging to 5 classes.


In [7]:
eyes_disease_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))


94765736/94765736 [==============================] - 0s 0us/step


In [8]:
for layer in eyes_disease_model.layers:
    layer.trainable = False

In [9]:
x = GlobalAveragePooling2D()(eyes_disease_model.output)
x = Dense(512, activation='relu')(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

In [10]:
model = Model(inputs=eyes_disease_model.input, outputs=predictions)

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [13]:
model_fitting = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.n // BATCH_SIZE
)

Epoch 1/20
20/20 [==============================] - 88s 4s/step - loss: 1.6567 - accuracy: 0.3569 - val_loss: 1.4689 - val_accuracy: 0.3542
Epoch 2/20
20/20 [==============================] - 6s 317ms/step - loss: 1.4580 - accuracy: 0.4437 - val_loss: 1.5746 - val_accuracy: 0.2083
Epoch 3/20
20/20 [==============================] - 6s 302ms/step - loss: 1.4755 - accuracy: 0.4469 - val_loss: 1.4441 - val_accuracy: 0.4583
Epoch 4/20
20/20 [==============================] - 5s 249ms/step - loss: 1.4876 - accuracy: 0.4212 - val_loss: 1.4284 - val_accuracy: 0.5000
Epoch 5/20
20/20 [==============================] - 6s 306ms/step - loss: 1.4640 - accuracy: 0.3987 - val_loss: 1.4405 - val_accuracy: 0.4583
Epoch 6/20
20/20 [==============================] - 5s 263ms/step - loss: 1.4567 - accuracy: 0.4309 - val_loss: 1.5300 - val_accuracy: 0.3542
Epoch 7/20
20/20 [==============================] - 6s 310ms/step - loss: 1.3621 - accuracy: 0.4309 - val_loss: 1.4478 - val_accuracy: 0.4167
Epoch 8/

In [14]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

4/4 [==============================] - 1s 370ms/step - loss: 1.5409 - accuracy: 0.4107
Test Loss: 1.5409129858016968
Test Accuracy: 0.4107142984867096


In [15]:
model.save('/content/drive/MyDrive/archive')

In [17]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.models import load_model

In [18]:
img_path = '/content/drive/MyDrive/archive/download.jpg'
img = image.load_img(img_path, target_size=(224, 224))

# Preprocess the input image
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

# Perform prediction
predictions = model.predict(img_array)

1/1 [==============================] - 2s 2s/step


In [19]:
print(predictions)

[[5.3103277e-10 2.0451262e-12 1.0000000e+00 4.7499644e-09 1.6535958e-12]]


In [20]:

class_labels = ['bulging', 'cataract', 'crossed', 'uvetis', 'glaucoma']

In [21]:
predicted_class_index = np.argmax(predictions)

predicted_class_label = class_labels[predicted_class_index]

print("Predicted class:", predicted_class_label)

Predicted class: crossed


In [23]:
from sklearn.metrics import classification_report, confusion_matrix

# Get true labels and predicted labels for the validation set
true_labels = validation_generator.classes
predicted_labels = model.predict(validation_generator).argmax(axis=1)

# Compute and print confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print("Confusion Matrix:")
print(conf_matrix)

# Compute and print classification report
class_report = classification_report(true_labels, predicted_labels, target_names=class_labels)
print("\nClassification Report:")
print(class_report)

4/4 [==============================] - 1s 67ms/step
Confusion Matrix:
[[ 0  0  2  2  0]
 [ 0  0  7  0  0]
 [ 0  0 25  1  0]
 [ 0  0 11  1  0]
 [ 0  0  7  0  0]]

Classification Report:
              precision    recall  f1-score   support

     bulging       0.00      0.00      0.00         4
    cataract       0.00      0.00      0.00         7
     crossed       0.48      0.96      0.64        26
      uvetis       0.25      0.08      0.12        12
    glaucoma       0.00      0.00      0.00         7

    accuracy                           0.46        56
   macro avg       0.15      0.21      0.15        56
weighted avg       0.28      0.46      0.32        56



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
